In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
from tensorflow.keras.models import load_model


In [5]:
## Load the trained model, scaler, and label encoder
model=load_model('model.h5')

## Load the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as f:
    onehot_encoder_geo = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [8]:
## Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 5,
    'Balance': 50000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}

In [9]:
## One-hot encode the Geography feature
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


e:\AI_ML Projects\ANN CLASSIFICATION\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,35,5,50000,2,1,1,60000


In [12]:
## Encode categorical features
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,35,5,50000,2,1,1,60000


In [13]:
## concatenate the one-hot encoded Geography with the input DataFrame
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,35,5,50000,2,1,1,60000,1.0,0.0,0.0


In [14]:
## scale the numerical features
input_scaled = scaler.transform(input_df)

In [15]:
input_scaled

array([[-0.53598516,  0.91324755, -0.37056859, -0.00134472, -0.41792108,
         0.80843615,  0.64920267,  0.97481699, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
## predict the churn probability
prediction = model.predict(input_scaled)

1/1 [==============================] - 0s 141ms/step


In [17]:
prediction
prediction_proba = prediction[0][0]

In [18]:
if prediction_proba > 0.5:
    print(f"The model predicts that the customer is likely to churn with a probability of {prediction_proba:.2f}.")
else:
    print(f"The model predicts that the customer is unlikely to churn with a probability of {1 - prediction_proba:.2f}.")

The model predicts that the customer is unlikely to churn with a probability of 0.99.
